In [10]:
import numpy as np
import scipy as sp
import scipy.constants as sc 
import scipy.fftpack as fft
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
import sys,os; sys.path.insert(0, os.path.expanduser('./lib_CUWA/')); 
import lib_CUWA_core as lib_CUWA #This library contains minimal code needed to run CUWA
import gaussian_beam as GB
from tqdm import tqdm

In [5]:
type(np.array([1]))

numpy.ndarray

In [2]:
def delay(time : np.ndarray, sig : np.ndarray) -> float : 
    """
    delay return the time delay of the signal

    Parameters
    ----------
    time : np.ndarray
        _description_
    sig : np.ndarray
        _description_

    Returns
    -------
    float
        _description_
    """    
    return time[np.argmax(abs(sig))]

In [7]:
def delay_dispersion(time, signal) -> float: 
    """
    delay_dispersion return the dispersion of the signal given a threshold 1/e * max(signal)

    Parameters
    ----------
    time : np.ndarray
        _description_
    signal : np.ndarray
        _description_

    Returns
    -------
    float
        _description_
    """    
    mask = np.abs(signal) > np.exp(-1) * np.max(np.abs(signal))
    time_window = time[mask]
    
    return (time_window[-1] - time_window[0]) / 2 

In [9]:
def Gaussian_turb(lx : float,ly : float,nx : float, ny : int, dx : float) -> np.ndarray: 
    """
    Gaussian_turb generate a 2d Gaussian turbulence field

    Parameters
    ----------
    lx : float
        correlation length in x of turbulence
    ly : float
        correlation length in y of turbulence
    nx : float
        number of points in x
    ny : int
        number of points in y
    dx : float
        grid spacing

    Returns
    -------
    np.ndarray
        _description_
    """    
    
    kx = 2*np.pi*np.arange(-0.5/dx,0.5/dx-0.5/dx/nx,1/dx/nx)
    ky = 2*np.pi*np.arange(-0.5/dx,0.5/dx-0.5/dx/ny,1/dx/ny)
    
    expx = np.exp(-kx**2*lx**2/8)
    expy = np.exp(-ky**2*ly**2/8)
    
    spectra = np.exp(np.random.rand(nx,ny)*2j*np.pi)
    spectra = spectra*expx[:,None]*expy[None,:]
    
    fluct = fft.ifftshift(fft.ifft2(fft.ifftshift(spectra)))
    
    return fluct.real() * np.pi / dx**2 * lx * ly

In [ ]:
def run_CUWA(C1 : lib_CUWA.CUWA ,w_p : float, w_c : float, b_x, b_y, b_z, beam : GB.Gaussian_beam,numplots,path) -> None:
    
    C1.set_plasma_data(w_p, w_c, b_x, b_y, b_z) # TODO ask for the meaning of the parameters b_x, b_y, b_z
    C1.set_antenna(beam, source='True', receiver='True')
    #C1.set_antenna(beam2, source='False', receiver='True')
    C1.run_init()
    progress_status = 0; st = 0;
    steps=round(C1.suggested_n_steps*1.7)
    reAz_list=[];imAz_list=[];t_list = [];IQ=[] #preparing list for the signal
    record=1 #recognizing the arrival of the pulse

    
    for t in tqdm(range(int(steps))):
        
        C1.run_source(t,beam)
        C1.run_phasor(t,beam)
        reAx_g,reAy_g,reAz_g,imAx_g,imAy_g,imAz_g = [field.get() for field in beam.A_g]
        C1.run_step()


        if numplots> 0 and (t+1) % np.floor(steps/numplots) == 0:
            
            fig   = plt.figure(figsize=(10*ratio,10))
            plt.contourf(X[0],Y[0],C1.w_p[0],30,cmap='cividis')
            plt.contour(X[0],Y[0],C1.get_magE()[0], [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],cmap='plasma')
            fig.savefig(path+'E_{0}.png'.format(t+1),dpi=400)
            plt.close()
            

        
    
        if t % (beam.T//8) == 0 and t> 1*beam.T:             
        
            if record == 1:   #recording the previous period signal
                t_list.append(t*C1.dt)
                IQ.append(np.sum(reAz_list)+1j*np.sum(imAz_list))
                #reAz_list=[]  #resetting the recording
                #imAz_list=[]
        
    
        if record==1:   #recording data for each period 
            reAz_list.append(np.trapz(reAz_g[0,0,:]))   
            imAz_list.append(np.trapz(imAz_g[0,0,:]))    
            if len(reAz_list)>beam.T:
                reAz_list.pop(0)
                imAz_list.pop(0)
    
    return t_list, IQ
